In [7]:
import pandas as pd
import pickle
import os

In [1]:
#Prof example install
#%pip install lda

  Created wheel for lda: filename=lda-2.0.0-cp38-cp38-win_amd64.whl size=346338 sha256=9f1a44fcee7b2548716e469505228f906a751b0e584cd06bdcaf804a19283a87
  Stored in directory: c:\users\louis\appdata\local\pip\cache\wheels\bf\64\c8\595c54dbccb82a2c7e46b0759030acc8770781a6ec547e0c53
Successfully built lda
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Prof example import 
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import lda
import pandas as pd
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df = pd.read_csv("20k_songs_and_lyrics.csv")

In [5]:
df.head()

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,track_instrumentalness,track_liveness,track_valence,track_tempo,track_release_date,artist_name,artist_genres,artist_followers,artist_popularity,lyrics
0,5h9hkYIBWTZYFRmF6Iz1gt,79dL7FLiJFOO0EoehUHQBv,5INjqkS1o8h1imAzPqGZBb,185773,True,https://api.spotify.com/v1/tracks/5h9hkYIBWTZY...,Love/Paranoia,63,track,0.502,...,0.000000,0.1710,0.354,111.024,2015-07-17,Tame Impala,"['australian psych', 'neo-psychedelic']",5272459,80,I may not be as honest as I ought to be\nNow t...
1,1mP8BJQBRZWjSQLVmxQEyG,7oUQQI3USlnBTUeN6KXXoo,5IDs1CK15HegSAhGEbSYXo,292680,False,https://api.spotify.com/v1/tracks/1mP8BJQBRZWj...,There'll Be Sad Songs (To Make You Cry),52,track,0.563,...,0.000000,0.1030,0.259,79.738,1986,Billy Ocean,"['disco', 'mellow gold', 'motown', 'new romant...",782975,65,"Sometimes I wonder, by the look in your eye\nW..."
2,3K7Q9PHUWPTaknlbFPThn2,2kc4mhFRsoIRVD0XEYnwhI,3VNITwohbvU5Wuy5PC6dsI,298866,False,https://api.spotify.com/v1/tracks/3K7Q9PHUWPTa...,Celebration,73,track,0.885,...,0.048300,0.0767,0.906,121.632,1980-09-29,Kool & The Gang,"['disco', 'funk', 'motown', 'post-disco', 'qui...",1826724,69,Yahoo! (This is your celebration)\nYahoo! (Thi...
3,123RhEzztpTieeEd13NexC,0NufsuTuf3U0BY0p6jFdxV,67ea9eGLXYMsO2eYQRui3w,162760,False,https://api.spotify.com/v1/tracks/123RhEzztpTi...,Pictures Of Lily,32,track,0.524,...,0.000000,0.1590,0.794,126.846,1971-10-30,The Who,"['album rock', 'art rock', 'blues rock', 'brit...",4286190,71,I used to wake up in the morning\nI used to fe...
4,5DJNyvuMmZfsfvdTpMMmUq,08IW6sUfN9fRYG8Xlgm4oC,1ShZZUjkbXCjhwrb18BA8I,252546,True,https://api.spotify.com/v1/tracks/5DJNyvuMmZfs...,Sour Patch Kids,70,track,0.523,...,0.000017,0.2190,0.581,76.588,2014-04-22,Bryce Vine,"['electropop', 'pop', 'pop rap']",431943,71,"When I was growing up, I had a lot of dreams, ..."


In [6]:
len(df)

16762

In [7]:
df_lyrics = pd.read_pickle("tracks_lyrics.pickle")

In [8]:
df_lyrics.head()

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,artist_name,artist_genres,artist_followers,artist_popularity,lyrics,lyrics_clean,lyrics_clean_split,lyrics_wordlen,final_genres,final_genre
0,5h9hkYIBWTZYFRmF6Iz1gt,79dL7FLiJFOO0EoehUHQBv,5INjqkS1o8h1imAzPqGZBb,185773,True,https://api.spotify.com/v1/tracks/5h9hkYIBWTZY...,Love/Paranoia,63,track,0.502,...,Tame Impala,"[australian psych, neo-psychedelic]",5272459,80,I may not be as honest as I ought to be\nNow t...,may honest ought walls said worry hit like arr...,"[may, honest, ought, walls, said, worry, hit, ...",83,"[other_genres, other_genres]",other_genres
1,1mP8BJQBRZWjSQLVmxQEyG,7oUQQI3USlnBTUeN6KXXoo,5IDs1CK15HegSAhGEbSYXo,292680,False,https://api.spotify.com/v1/tracks/1mP8BJQBRZWj...,There'll Be Sad Songs (To Make You Cry),52,track,0.563,...,Billy Ocean,"[disco, mellow gold, motown, new romantic, new...",782975,65,"Sometimes I wonder, by the look in your eye\nW...",sometimes wonder look eye standing besides fev...,"[sometimes, wonder, look, eye, standing, besid...",141,"[disco, other_genres, other_genres, other_genr...",disco
2,3K7Q9PHUWPTaknlbFPThn2,2kc4mhFRsoIRVD0XEYnwhI,3VNITwohbvU5Wuy5PC6dsI,298866,False,https://api.spotify.com/v1/tracks/3K7Q9PHUWPTa...,Celebration,73,track,0.885,...,Kool & The Gang,"[disco, funk, motown, post-disco, quiet storm,...",1826724,69,Yahoo! (This is your celebration)\nYahoo! (Thi...,yahoo celebration yahoo celebration celebrate ...,"[yahoo, celebration, yahoo, celebration, celeb...",196,"[disco, other_genres, other_genres, disco, oth...",disco
3,123RhEzztpTieeEd13NexC,0NufsuTuf3U0BY0p6jFdxV,67ea9eGLXYMsO2eYQRui3w,162760,False,https://api.spotify.com/v1/tracks/123RhEzztpTi...,Pictures Of Lily,32,track,0.524,...,The Who,"[album rock, art rock, blues rock, british inv...",4286190,71,I used to wake up in the morning\nI used to fe...,used wake morning used feel bad got sick sleep...,"[used, wake, morning, used, feel, bad, got, si...",102,"[rock, rock, blues, other_genres, rock, rock, ...",blues
4,5DJNyvuMmZfsfvdTpMMmUq,08IW6sUfN9fRYG8Xlgm4oC,1ShZZUjkbXCjhwrb18BA8I,252546,True,https://api.spotify.com/v1/tracks/5DJNyvuMmZfs...,Sour Patch Kids,70,track,0.523,...,Bryce Vine,"[electropop, pop, pop rap]",431943,71,"When I was growing up, I had a lot of dreams, ...",growing lot dreams momma told son could anythi...,"[growing, lot, dreams, momma, told, son, could...",251,"[pop, pop, rap]",rap


In [10]:
word_tokenizer = RegexpTokenizer(r'\w+')

wordnet_lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

stopwords_nltk = set(stopwords.words('english'))



In [12]:
temp = df_lyrics["lyrics_clean"]

In [ ]:
def tokenize_text(raw_text):
    text = raw_text.lower()
    tokens = word_tokenizer.tokenize(text)
    return tokens

In [ ]:
def tokenize_text_lemma(raw_text):
    lowercase = raw_text.lower()
    text = wordnet_lemmatizer.lemmatize(lowercase)
    tokens = word_tokenizer.tokenize(text)
    return tokens

In [ ]:
def tokenize_text_stem(raw_text):
    lowercase = raw_text.lower()
    text = wordnet_stemmer.lemmatize(lowercase)
    tokens = word_tokenizer.tokenize(text)
    return tokens